In [2]:
from pylab import *
from sympy import *
from sympy.solvers import solve
import transforms3d


init_printing()

## Initiatlisation des paramèters : 

In [3]:
m = Symbol('m', real=True)                                                         # Masse du drone

omega1, omega2, omega3 = symbols('\omega_1, \omega_2, \omega_3', real=True)
Omega         = Matrix([omega1, omega2, omega3])                                    # Vecteur vitesse de rotation
 
Vx, Vy, Vz   = symbols('V_x V_y V_z',real=True)
V          = Matrix([Vx, Vy, Vz])                                             # Vitesse du drone dans le repère NED

J11, J22, J33 = symbols('J_{11}, J_{22}, J_{33}', real=True)
J             = Matrix([[J11,0,0],[0,J22,0],[0,0,J33]])
J             = Matrix([diag(J11, J22, J33)])                    # Matrice D'inertie

Fx_rotor, Fy_rotor, Fz_rotor   = symbols('F_{x_{rotor}}  F_{y_{rotor}}  F_{z_{rotor}}',real=True)
F_rotor = Matrix([Fx_rotor, Fy_rotor, Fz_rotor])

Qx_rotor, Qy_rotor, Qz_rotor   = symbols('Q_{x_{rotor}}, Q_{y_{rotor}}, Q_{z_{rotor}}',real=True)
Q_rotor = Matrix([Qx_rotor, Qy_rotor, Qz_rotor])

Fx_wing, Fy_wing, Fz_wing   = symbols('F_{x_{wing}}, F_{y_{wing}}, F_{z_{wing}}',real=True)
F_wing = Matrix([Fx_wing, Fy_wing, Fz_wing])

Qx_wing, Qy_wing, Qz_wing   = symbols('Q_{x_{wing}}, Q_{y_{wing}}, Q_{z_{wing}}',real=True)
Q_wing = Matrix([Qx_wing, Qy_wing, Qz_wing])

## Equation de la dynamique

In [4]:
def Dynamique(sum_force_rotor, sum_force_wing, sum_torque_rotor, sum_torque_wing, m, J, Omega):
    Accel = (sum_force_rotor + sum_force_wing) / m
    OmegaPoint = (sum_torque_rotor + sum_torque_wing - Omega.cross(J * Omega))
    OmegaPoint = Matrix([OmegaPoint[0] / J[0], OmegaPoint[1] / J[4], OmegaPoint[2] / J[8]])
    return Accel, OmegaPoint

In [8]:
Acc, Op = Dynamique(F_rotor, F_wing, Q_rotor, Q_wing, m, J, Omega)
Acc
Acc

⎡F_{x_{rotor}} + F_{x_{wing}}⎤
⎢────────────────────────────⎥
⎢             m              ⎥
⎢                            ⎥
⎢F_{y_{rotor}} + F_{y_{wing}}⎥
⎢────────────────────────────⎥
⎢             m              ⎥
⎢                            ⎥
⎢F_{z_{rotor}} + F_{z_{wing}}⎥
⎢────────────────────────────⎥
⎣             m              ⎦

In [6]:
Op.simplify()
Op

⎡J_{22}⋅\omega₂⋅\omega₃ - J_{33}⋅\omega₂⋅\omega₃ + Q_{x_{rotor}} + Q_{x_{wing}
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                    J_{11}                                   
⎢                                                                             
⎢-J_{11}⋅\omega₁⋅\omega₃ + J_{33}⋅\omega₁⋅\omega₃ + Q_{y_{rotor}} + Q_{y_{wing
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                     J_{22}                                  
⎢                                                                             
⎢J_{11}⋅\omega₁⋅\omega₂ - J_{22}⋅\omega₁⋅\omega₂ + Q_{z_{rotor}} + Q_{z_{wing}
⎢─────────────────────────────────────────────────────────────────────────────
⎣                                    J_{33}                                   

} ⎤
─ ⎥
  ⎥
  ⎥
}}⎥
──⎥
  ⎥
  ⎥
} ⎥
─ ⎥
  ⎦

In [7]:
Acc_function = lambdify((F_rotor, F_wing, m), Acc)
OmegaP_function = lambdify((Q_rotor, Q_wing, J11, J22, J33, Omega), Op)

In [31]:
from time import *
from threading import get_ident
Id = pthread_getcpuclockid(get_ident())
T_test_1 =0
T_test_2 = 0
J11 = 0.19
J22 = 0.15
J33 = 0.15
J=diag(J11,J22,J33)
Q_rotor=Matrix([0,0,1])
Q_wing=Matrix([0,1,0])
F_rotor=Matrix([1,0,0])
F_wing=Matrix([1,0,0])
P = 10000
for i in range(P):
    t1_test_1 = clock_gettime(Id)
    Acc1 = Acc_function(F_rotor, F_wing, m)
    Acc_ang = OmegaP_function(Q_rotor, Q_wing, J11, J22, J33, Omega)
    t2_test_1 =clock_gettime(Id)
    T_test_1 += t2_test_1 - t1_test_1


    t1_test_2 = clock_gettime(Id)
    Acc, Op = Dynamique(F_rotor, F_wing, Q_rotor, Q_wing, m, J, Omega)
    t2_test_2 = clock_gettime(Id)
    T_test_2 += t2_test_2 - t1_test_2

if T_test_1 < T_test_2 : print("le mode lambda function est plus rapide")
else: print('le mode function classique est plus rapide')
print(T_test_1/P, T_test_2/P)

le mode lambda function est plus rapide
0.00024787263579999073 0.0002481064393000157
